Pobierz dane dotyczące scoringu kredytowego

*Pracujesz jako analityk danych w globalnej firmie finansowej. Przez lata firma gromadziła podstawowe dane bankowe i wiele informacji związanych z kredytami. Kierownictwo chce zbudować inteligentny system do segregowania ludzi w przedziałach oceny kredytowej, aby zmniejszyć wysiłek ręczny.*

**Zadanie na dziś: przygotować dane do modelu**

In [89]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

import danych

In [137]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train.csv"

# Load the latest version
df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,
)

print("First 5 records:", df.head())

<ipython-input-137-db52f5663265>:9: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(KaggleDatasetAdapter.PANDAS, "parisrohan/credit-score-classification", file_path,


First 5 records:        ID Customer_ID     Month           Name   Age          SSN Occupation  \
0  0x1602   CUS_0xd40   January  Aaron Maashoh    23  821-00-0265  Scientist   
1  0x1603   CUS_0xd40  February  Aaron Maashoh    23  821-00-0265  Scientist   
2  0x1604   CUS_0xd40     March  Aaron Maashoh  -500  821-00-0265  Scientist   
3  0x1605   CUS_0xd40     April  Aaron Maashoh    23  821-00-0265  Scientist   
4  0x1606   CUS_0xd40       May  Aaron Maashoh    23  821-00-0265  Scientist   

  Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  ...  Credit_Mix  \
0      19114.12            1824.843333                  3  ...           _   
1      19114.12                    NaN                  3  ...        Good   
2      19114.12                    NaN                  3  ...        Good   
3      19114.12                    NaN                  3  ...        Good   
4      19114.12            1824.843333                  3  ...        Good   

   Outstanding_Debt Credit_Utiliz

/usr/local/lib/python3.11/dist-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


**Poznaj dane:**

1.   sprawdź wymiary i typy kolumn
2.   usuń duplikaty
3.   usuń zdbędne kolumny
4.   sprawdz liczbę braków danych w poszczególnych kolumnach






In [138]:
# 1 - sprawdzanie wymiarów i typów kolumn
wiersze, kolumny = df.shape
print(f"Wiersze: {wiersze}\nKolumny {kolumny}")

df.info()

Wiersze: 100000
Kolumny 28
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan      

In [139]:
# 2 -usuwanie duplikatów
df = df.drop_duplicates()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

Usuń kolumny, które nie będą wykorzystywane w analizie ['ID','Customer_ID','SSN','Name']

In [140]:
# 3 - usuwanie zbędnych kolumn
df = df.drop(columns = ['ID','Customer_ID','SSN','Name'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  object 
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  object 
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  object 
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    92998 non-null   object 
 12  Changed_Credit_Limit      100000 non-null  object 
 13  Num_Credit_Inquiries      98035 non-null   fl

In [141]:
df['Credit_History_Age']

,Credit_History_Age
0,22 Years and 1 Months
1,NaN
2,22 Years and 3 Months
3,22 Years and 4 Months
4,22 Years and 5 Months
...,...
99995,31 Years and 6 Months
99996,31 Years and 7 Months
99997,31 Years and 8 Months
99998,31 Years and 9 Months


usuń niektore z zbędnych znakow ('_','-'), zmień puste komorki na nan





In [142]:
for element in df.columns:
  df[element] = df[element].astype(str)
  df[element] = df[element].str.replace('-', '')
  df[element] = df[element].str.replace('_', '')
  df[element] = df[element].replace('', np.nan)
  df[element] = df[element].replace('nan', np.nan)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Month                     100000 non-null  object
 1   Age                       100000 non-null  object
 2   Occupation                92938 non-null   object
 3   Annual_Income             100000 non-null  object
 4   Monthly_Inhand_Salary     84998 non-null   object
 5   Num_Bank_Accounts         100000 non-null  object
 6   Num_Credit_Card           100000 non-null  object
 7   Interest_Rate             100000 non-null  object
 8   Num_of_Loan               100000 non-null  object
 9   Type_of_Loan              88592 non-null   object
 10  Delay_from_due_date       100000 non-null  object
 11  Num_of_Delayed_Payment    92998 non-null   object
 12  Changed_Credit_Limit      97909 non-null   object
 13  Num_Credit_Inquiries      98035 non-null   object
 14  Credi

In [143]:
df['Month']

,Month
0,January
1,February
2,March
3,April
4,May
...,...
99995,April
99996,May
99997,June
99998,July


Zmień nazwy miesiecy na zmienne liczbowe zgodnie z kolejnością miesięcy

In [144]:
month_mapping = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12
}

df['Month'] = df["Month"].map(month_mapping)

df["Month"]

,Month
0,1
1,2
2,3
3,4
4,5
...,...
99995,4
99996,5
99997,6
99998,7


Zakoduj zmienne tekstowe: 'Occupation', 'Type_of_Loan', 'Payment_Behaviour', 'Credit_Mix', użyj LabelEncodera

In [145]:
le = LabelEncoder()
df['Occupation'] = le.fit_transform(df['Occupation'])
df['Payment_Behaviour'] = le.fit_transform(df['Payment_Behaviour'])
df['Credit_Mix'] = le.fit_transform(df['Credit_Mix'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Month                     100000 non-null  int64 
 1   Age                       100000 non-null  object
 2   Occupation                100000 non-null  int64 
 3   Annual_Income             100000 non-null  object
 4   Monthly_Inhand_Salary     84998 non-null   object
 5   Num_Bank_Accounts         100000 non-null  object
 6   Num_Credit_Card           100000 non-null  object
 7   Interest_Rate             100000 non-null  object
 8   Num_of_Loan               100000 non-null  object
 9   Type_of_Loan              88592 non-null   object
 10  Delay_from_due_date       100000 non-null  object
 11  Num_of_Delayed_Payment    92998 non-null   object
 12  Changed_Credit_Limit      97909 non-null   object
 13  Num_Credit_Inquiries      98035 non-null   object
 14  Credi

Przelicz kolumne Credit_History_Age na liczbę mięsiecy








In [147]:
df['Credit_History_Age']

def convert(data):
  if pd.isna(data):
    return data
  data_split = data.split(' ')
  years = int(data_split[0])
  months = int(data_split[3])
  return_value = years*12 + months
  return return_value

df['Credit_History_Age'] = df['Credit_History_Age'].apply(convert)

df['Credit_History_Age']



,Credit_History_Age
0,265.0
1,NaN
2,267.0
3,268.0
4,269.0
...,...
99995,378.0
99996,379.0
99997,380.0
99998,381.0


In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  int64  
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  int64  
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     84998 non-null   object 
 5   Num_Bank_Accounts         100000 non-null  object 
 6   Num_Credit_Card           100000 non-null  object 
 7   Interest_Rate             100000 non-null  object 
 8   Num_of_Loan               100000 non-null  object 
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  object 
 11  Num_of_Delayed_Payment    92998 non-null   object 
 12  Changed_Credit_Limit      97909 non-null   object 
 13  Num_Credit_Inquiries      98035 non-null   ob

In [149]:
# Oczyszczanie pola Type_of_Loan
df['Type_of_Loan'] = df['Type_of_Loan'].fillna('')
df['Type_of_Loan'] = df['Type_of_Loan'].str.replace(r'\band\b', '', regex=True).str.strip() #removes "and"

# Split and clean the values
df_type_of_loan = df['Type_of_Loan'].str.split(r',\s*')  # Handles different spacing

# Use MultiLabelBinarizer
mlb = MultiLabelBinarizer()
dt = mlb.fit_transform(df_type_of_loan.to_list())  # Ensure correct format

# Convert to DataFrame
encoded_df = pd.DataFrame(data=dt, columns=mlb.classes_)

# Merge with original DataFrame and drop old column
df = df.join(encoded_df).drop(columns=['Type_of_Loan'])

print(df.head())

   Month  Age  Occupation Annual_Income Monthly_Inhand_Salary  \
0      1   23          12      19114.12    1824.8433333333328   
1      2   23          12      19114.12                   NaN   
2      3  500          12      19114.12                   NaN   
3      4   23          12      19114.12                   NaN   
4      5   23          12      19114.12    1824.8433333333328   

  Num_Bank_Accounts Num_Credit_Card Interest_Rate Num_of_Loan  \
0                 3               4             3           4   
1                 3               4             3           4   
2                 3               4             3           4   
3                 3               4             3           4   
4                 3               4             3           4   

  Delay_from_due_date  ...    Auto Loan CreditBuilder Loan  \
0                   3  ...  0         1                  1   
1                   1  ...  0         1                  1   
2                   3  ...  0   

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 33 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Month                     100000 non-null  int64  
 1   Age                       100000 non-null  object 
 2   Occupation                100000 non-null  int64  
 3   Annual_Income             100000 non-null  object 
 4   Monthly_Inhand_Salary     84998 non-null   object 
 5   Num_Bank_Accounts         100000 non-null  object 
 6   Num_Credit_Card           100000 non-null  object 
 7   Interest_Rate             100000 non-null  object 
 8   Num_of_Loan               100000 non-null  object 
 9   Delay_from_due_date       100000 non-null  object 
 10  Num_of_Delayed_Payment    92998 non-null   object 
 11  Changed_Credit_Limit      97909 non-null   object 
 12  Num_Credit_Inquiries      98035 non-null   object 
 13  Credit_Mix                100000 non-null  in

Zmien inne zmienne na numeryczne

Uzupełnij braki w kolumnie Monthly_Inhand_Salary,  Annual_Income, .. średnią

Narysuj rozkłady zmiennych 'Annual_Income', 'Monthly_Balance'

usuń wartości odstające (w przypadku wieku wszystkie ujemne i te powyzej 100 lat)

In [151]:
df['Age'] = df['Age'].astype(int)
df["Age"] = df["Age"].apply(lambda x: np.nan if x < 0 or x > 100 else x)
df["Age"]

,Age
0,23.0
1,23.0
2,NaN
3,23.0
4,23.0
...,...
99995,25.0
99996,25.0
99997,25.0
99998,25.0


Zwizualizuj korelacje między zmiennymi